In [1]:
!pip install openrouteservice

In [2]:
import openrouteservice

coords = ((8.34234,48.23424),(8.34423,48.26424))

client = openrouteservice.Client(key='5b3ce3597851110001cf62485b9d93cb553447258f757abe599113ab') # Specify your personal API key
routes = client.directions(coords)

print(routes)

{'bbox': [8.327725, 48.231953, 8.345244, 48.263552], 'metadata': {'attribution': 'openrouteservice.org | OpenStreetMap contributors', 'service': 'routing', 'timestamp': 1700654412128, 'query': {'coordinates': [[8.34234, 48.23424], [8.34423, 48.26424]], 'profile': 'driving-car', 'format': 'json'}, 'engine': {'version': '7.1.0', 'build_date': '2023-07-09T01:31:50Z', 'graph_date': '2023-11-19T15:25:49Z'}}, 'routes': [{'summary': {'distance': 5477.4, 'duration': 803.6}, 'segments': [{'distance': 5477.4, 'duration': 803.6, 'steps': [{'distance': 247.2, 'duration': 59.3, 'type': 11, 'instruction': 'Head south on Benatweg', 'name': 'Benatweg', 'way_points': [0, 8]}, {'distance': 51.8, 'duration': 12.4, 'type': 6, 'instruction': 'Continue straight onto Benatweg', 'name': 'Benatweg', 'way_points': [8, 10]}, {'distance': 1684.0, 'duration': 134.7, 'type': 1, 'instruction': 'Turn right onto Hölzle, K 5528', 'name': 'Hölzle, K 5528', 'way_points': [10, 64]}, {'distance': 263.7, 'duration': 31.6, '

In [3]:
import requests

# Substitua a URL abaixo pela URL do seu serviço OSRM
url = 'http://router.project-osrm.org/route/v1/driving/'

# Substitua as coordenadas pelos pontos que você deseja rotear
coordinates = '13.388860,52.517037;13.397634,52.529407;13.428555,52.523219'

# Construa a URL completa
full_url = f'{url}{coordinates}'

# Faça a requisição HTTP
response = requests.get(full_url)

# Verifique se a requisição foi bem-sucedida (código de status HTTP 200)
if response.status_code == 200:
    # Converta a resposta para JSON
    data = response.json()
    # Faça o que quiser com os dados da rota
    print(data['routes'][0]['legs'][0]['duration'])
else:
    # Imprima uma mensagem de erro se a requisição falhar
    print(f'Erro na requisição: {response.status_code}')
    print(response.text)

260.3


In [4]:
from math import radians, sin, cos, sqrt, atan2

def haversine_distance(coord1, coord2):
    R = 6371.0

    # Converte as coordenadas de graus para radianos
    lat1, lon1 = radians(coord1[1]), radians(coord1[0])
    lat2, lon2 = radians(coord2[1]), radians(coord2[0])

    # Diferença nas coordenadas
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Fórmula de Haversine
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Distância em quilômetros
    distance = R * c

    return distance

coords = ((8.34234, 48.23424), (8.34423, 48.26424))

distance = haversine_distance(coords[0], coords[1])

print(f'A distância entre os pontos é {distance}')

A distância entre os pontos é 3.338781888814991


In [5]:
import pandas as pd

df_curitiba_tecnicos = pd.read_csv('/content/Ponto de Partida_Tecnicos-2023-11-06-20-00-15.xlsx - Tecnicos Universitarios (1).csv')
df_curitiba_inicial = pd.read_csv('/content/servicos_setembro_pr_curitiba.txt') 

In [7]:
df_curitiba_inicial['dt_abertura_ba'] = pd.to_datetime(df_curitiba_inicial['dt_abertura_ba'])
novo_dataset = df_curitiba_inicial[df_curitiba_inicial['dt_abertura_ba'].dt.date == pd.to_datetime('2023-09-15').date()]

In [9]:
df_curitiba_tecnicos = df_curitiba_tecnicos.query("Cidade == 'Curitiba'")
df_tecnicos = df_curitiba_tecnicos[['Nome','Latitude', 'Longitude']]

#Resultados Output


In [10]:
#número de setores

df_pedidos.setor.unique().size

12

In [11]:
#quantidade de pedidos em cada setor

qtd_pedidos_setores = []
for setor in df_pedidos.setor.unique():
  qtd_pedidos_setores.append(len(df_pedidos.query(f"setor == '{setor}'")))

qtd_pedidos_setores

[17, 38, 31, 46, 35, 18, 12, 64, 7, 43, 30, 27]

In [12]:
#quantitade de técnicos

len(df_tecnicos)

125

In [17]:
#tempos de deslocamentos de tecnicos para cada pedidos
import requests
import numpy as np

num_rows = len(df_tecnicos)
num_cols = len(df_pedidos)
matriz_numeros_aleatorios = np.random.rand(num_rows, num_cols)

url = 'http://router.project-osrm.org/route/v1/driving/'

for i, setor in enumerate(df_pedidos['setor'].unique()):
    for j, pedido in enumerate(df_pedidos.query(f"setor == '{setor}'")):
        for k, tecnico in enumerate(df_tecnicos['Nome']):
          matriz_numeros_aleatorios[k, j] = haversine_distance((float(df_tecnicos.iloc[k].Latitude.replace(',', '.')), float(df_tecnicos.iloc[k].Longitude.replace(',', '.'))), (df_pedidos.iloc[j].latitude, df_pedidos.iloc[j].longitude))

          coordinates = f"{df_tecnicos.iloc[k].Longitude.replace(',', '.')},{df_tecnicos.iloc[k].Latitude.replace(',', '.')};{df_tecnicos.iloc[j].Longitude.replace(',', '.')},{df_tecnicos.iloc[j].Latitude.replace(',', '.')}"

          full_url = f'{url}{coordinates}'
          response = requests.get(full_url)

          if response.status_code == 200:
              data = response.json()
              matriz_numeros_aleatorios[k, j] = float(data['routes'][0]['legs'][0]['duration'])
              print(float(data['routes'][0]['legs'][0]['duration']))
          else:
              matriz_numeros_aleatorios[k, j] = haversine_distance((float(df_tecnicos.iloc[k].Latitude.replace(',', '.')), float(df_tecnicos.iloc[k].Longitude.replace(',', '.'))), (df_pedidos.iloc[j].latitude, df_pedidos.iloc[j].longitude))

df_resultado = pd.DataFrame(matriz_numeros_aleatorios, index=df_tecnicos['Nome'], columns=df_pedidos['id_venda'])
df_resultado

0.0
995.4
1400.2
1091.0
399.5
1314.0
667.0
1491.3
1025.6
564.7
948.9
1596.1
1256.2
1334.5
1240.2
1028.0
1279.5
1494.5
1348.5
444.6
865.8
1597.1
1421.1
1749.2
615.9
1142.0
1421.1
1481.2
1496.4
1620.5
1005.4
981.6
1242.1
1448.7
1053.4
1173.7
1040.8
1116.6
838.5
754.3
937.7
853.8
1205.0
313.3
249.9
853.8
1407.0
1458.6
1360.1
1503.8
1400.2
154.8
692.6
358.1
1129.1
1423.5
1430.9
1010.9
864.4
1570.1
1537.7
910.1
1303.4
818.9
1318.7
1368.5
1279.0
1465.1
1160.0
1628.7
897.9
638.4
1398.9
843.2
1253.6
1438.0
888.4
1044.5
1087.8
1152.8
1286.5
1335.2
538.5
1636.7
1620.0
1001.9
1142.0
1749.2
72.7
212.5
476.4
609.3
1173.7
1012.5
1133.7
477.1
235.3
868.6
1205.0
1173.7
937.7
665.2
1084.7
1077.4
1516.3
1326.2
1242.1
1692.9
1012.5
1197.7
1299.9
1001.9
1249.7
1444.1
1019.8
1447.7
1091.2
1010.5
965.2
290.3
839.7
1279.3
235.8
1287.4
1463.6
1054.1
0.0
530.4
1523.3
1024.7
444.2
1430.3
1056.6
268.3
964.7
123.0
1133.8
1202.3
418.0
416.9
284.3
966.7
779.9
1035.7
1031.3
781.4
1134.8
568.0
1379.1
1101.8
913.4
568

KeyboardInterrupt: ignored

array([9.86360389e+00, 7.97151369e+00, 4.11881510e+03, 2.71060404e+00,
       3.41215977e-01, 4.88108427e-02, 9.50527938e-01, 3.32159406e-01,
       2.72001196e-01, 3.24769017e-01, 2.91276968e-01, 3.01909436e-01,
       3.12384732e-01, 4.33980678e-01, 2.33079267e-01, 4.56123094e-01,
       8.45566394e-01, 3.27783549e-01, 6.50725805e-01, 2.68263489e-01,
       6.19665479e-01, 8.37303288e-01, 8.78418659e-01, 9.38430049e-01,
       6.23607340e-01, 1.87156451e-01, 1.06199370e-02, 3.30573203e-01,
       8.93847914e-02, 1.25678018e-01, 7.08680164e-02, 7.73450543e-01,
       6.45419282e-02, 9.11424146e-01, 7.61320200e-01, 7.57123388e-01,
       1.23033240e-01, 1.59713377e-02, 9.03751723e-01, 3.51118146e-01,
       5.47405865e-01, 1.39313255e-01, 6.30747783e-01, 2.13864413e-01,
       8.35008295e-01, 8.49136712e-01, 8.27518618e-01, 7.77423032e-01,
       7.01892640e-01, 6.43274010e-01, 7.44214207e-01, 2.77142295e-01,
       3.31102618e-01, 4.97861563e-01, 1.69267576e-01, 7.50972018e-01,
      